<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/Taranov-Anatoly-folder/Taranov%20Anatoly/2_1InspectClinicRSgr2_TA_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ТЕХНИЧЕСКОЕ ЗАДАНИЕ



Рекомендательная система для профпатолога (INSPECTRUM CLINIC)

Цель: Разработать прототип системы, реализующей функцию рекомендательной системы для врача профпатолога.

Принцип работы: Система должна помогать врачу вынести решение о том годен, не годен или ограниченно годен человек на определенной должности. В датасете целевым столбцом для предсказания нейросетью является столбец под названием: “ЗаключениеМК”. Система может дополнять вердикт мнением, почему человек не годен, или почему ограниченно годен.

Требования интерфейсу: Минимальный уровень реализации: notebook google colaboratory. Требуется реализовать простой веб-интерфейс для загрузки анализируемого моделью нейросети набора данных и удобным выводом результатов вынесенного решения.

Дополнительно: передача продвинутой системы веб-интерфейса, дополняющей вердикт мнением, почему человек не годен, или почему ограниченно годен.

Требования к тестированию, метрикам качества: Оценка результатов работы системы производится Заказчиком на тестовом наборе Заказчика. Метрика точности - точность рекомендации должна быть не ниже 95%

# Загрузка библиотек и датасета

In [ ]:
# Импорт библиотек и инструментов
from google.colab import files              # Импортируем библиотеку files для работы с файлами
import re                                   # Регулярные выражения
import pandas as pd                         # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt             # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import seaborn as sns                       # Импортируем модуль seaborn библиотеки matplotlib для построения графиков
import datetime                             # Импортируем модуль datetime для обработки данных в формете даты
import numpy as np                          # Импортируем библиотеку numpy

%matplotlib inline
import gdown
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

In [ ]:
                                            # Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/БазыCSV/InClinic/ВесьЗаПериод.xlsx')       # Загрузка файла из диска в colab

In [ ]:
#data = pd.read_csv('/content/drive/MyDrive/БазыCSV/InClinic/Весь период.csv', sep = ';', on_bad_lines='skip', engine='python' )

# Анализ формы и состава данных

В ходе анализа предоставленной базы данных выяснилось что достаточно большая часть содержит информацию не влияющую на выполнение задачи по техническому заданию. Принято решение создания датафрейма на позициях необходимых для решения задачи.


---

Запускать только при необходимости анализа

In [ ]:
# путь до папки, содержащей файлы (замените на ваш путь)
folder_path = '/content/drive/MyDrive/Работа в УИИ/Профпатолог стажировка (старички) 01.04/datasets/24.06'

# получаем список файлов в папке
file_list = os.listdir(folder_path)

# последовательно выводим содержимое каждого .csv файла
for file_name in file_list:
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        print("Contens of ", file_name, ":")
        try:
            df = pd.read_csv(file_path, sep=',')
        except:
            try:
                df = pd.read_csv(file_path, sep=';')
            except:
                print("Could not determine separator")
                continue
        display(df) # выводим содержимое файла в виде таблицы

In [ ]:
print(data.shape)                            # Форма и размер
print(data.size)                             # Колличество элементов
data.isna().sum().sum()                      # Вычислит общее количество пропусков во всей таблице
#data.info()                                 # Сводная информация
#data.head()                                 # Вывод первых 5 строк таблицы

(32686, 289)
9446254


8791393

In [ ]:
data.describe(include=['O'])                  # Статистика

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,Контрагент,Цех,...,ХирургНеИсп2_МКБ103,ХирургНеИсп3_МКБ101,ХирургНеИсп3_МКБ102,ХирургНеИсп3_МКБ103,ЭКГНеИсп1_МКБ101,ЭКГНеИсп1_МКБ102,ЭКГНеИсп2_МКБ101,ЭКГНеИсп2_МКБ102,Эндоскопия1_МКБ101,Эндоскопия1_МКБ102
count,32686,32686,32447,32682,32682,32686,32686,32686,32670,10596,...,6,6,6,6,864,22,22,22,94,6
unique,18318,16719,10265,3,3,468,2,2,610,750,...,1,1,1,1,1,1,1,1,9,2
top,b31f3c71-87d7-11ec-80fc-0cc47aab8067,a0a96ecb-6cd5-11ea-80dd-0cc47aab8067,1988-01-01T00:00:00,Мужской,Мужской,2023-05-31T00:00:00,Периодический,Периодический,14b3b113-2a96-11e9-80ce-0cc47aab8067,Без отдела,...,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,K86.1
freq,7,10,19,23122,23122,1755,32582,32582,2782,1964,...,6,6,6,6,864,22,22,22,42,4


In [ ]:
data.info([0,-1])                              # Анализ состава информации в столбцах

In [ ]:
# Выведем уникальные значения для данной колонки
#display(data['Профессия'].unique())

In [ ]:
# Индекс в таблице датафрейма 12
#n = 12
# Значения каждого столбца
#for i in range(len(data.values[n])):
#   print('{:>2}  {}'.format(i, data.values[i][n]))

In [ ]:
# Выведем уникальные значения для данной колонки
display(data['ЗаключениеМК'].unique())

array(['Годен', 'ГоденСКоррекциейЗрения',
       'НуждаетсяВДообследованииИЛечении', nan, 'ГоденБезРаботНаВысотах',
       'ГоденСКоррекциейСлуха', ' п. 11.3', 'ОграниченноГоден',
       'машинист бульдозера', 'Газоспасатель', ' Газоспасатель',
       'Негоден'], dtype=object)

# Отбор столбцов с целевыми данными

* Отбор столбцов - персона/профессия/статус:
"КлиентДатаРождения",  "КлиентПол",  "ТипМедосмотраПредставление", "ПсихОсвидетельствование",  "Цех", "Профессия", "ЗаключениеМК", "ЗаключениеМКПредставление",  "ДиспансернаяГруппаПредставление",
"ВредныеФакторы".

* Отбор столбцов по специализациям врачей
 с целевой информацией - Специализация/обследование/состояние:
"МКБ101",  "МКБ102", "МКБ103",  "МКБ104", " Представление", "ДиагнозПредставление",   "Диагноз_ЗначениеПредставление",
"ПрофнепригодностьВременнаяПредставление",  "ПрофнепригодностьВременнаяПредставление1", "ПрофнепригодностьВременная1","Анамнез_ЗначениеПредставление",  "Профессиональное1", "ЖалобыДополнение", "Рекомендация1", "Заключение".


In [ ]:
# Создание копии таблицы
df = data.copy()

In [ ]:
# Список столбцов с персональными данными
person = ['КлиентДатаРождения', 'КлиентПол', 'Профессия','ВредныеФакторы',
          'ПсихОсвидетельствование', 'ЗаключениеМК', 'ЗаключениеМКПредставление']
def person_columns(df):
    person = []
    for column in df.columns:
                columns_selected.append(df[column])
    return df[person]

person_dat = df[person]

In [ ]:
words = {"МКБ101", "МКБ102","МКБ103", "МКБ104",  "Представление",
         "ПрофнепригодностьВременнаяПредставление","ПрофнепригодностьВременнаяПредставление1",
         "ПрофнепригодностьВременная1","ЖалобыДополнение","Заключение"}
def find_columns(df, words):
    '''Функция для поиска столбцов в датафрейме df.Возвращаем:
     -новый датафрейм, состоящий только из целевых столбцов
    '''
    #разделяем названия столбцов на отдельные слова
    column_words = [s.split('_') for s in df.columns]
    # ищем вхождения каждого слова из списка words
    # в каждом списке слов из column_words
    word_indices = {}
    for i, cw in enumerate(column_words):
        for word in words:
            if word in cw:
                if i not in word_indices:
                    word_indices[i] = 1
                else:
                    word_indices[i] += 1

    #составляем список индексов столбцов, которые содержат
    #хотя бы одно искомое слово
    index_list = [i for i, count in word_indices.items() if count > 0]

    #возвращаем новый датафрейм, состоящий только из найденных столбцов
    return df.iloc[:, index_list]

diagnosis_dat = find_columns(df, words)


In [ ]:
# Объединяяем выборки столбцов
result = pd.concat([person_dat, diagnosis_dat], axis=1)
df1= result

In [ ]:
# Удаляем полностью пустые столбцы
df1.dropna(axis=1, how='all', inplace=True)
# Удаляем дубликаты столбцов
df1 = df1.loc[:,~df1.columns.duplicated()]
# Удаляем полностью пустые строки
df1.dropna(axis=0, how='all', inplace=True)
# Доля пропусков от общего количества элементов
df1.head(2).isna().sum() / df1.head(2).shape[0]
# Вычислим общее количество пропусков во всей таблице
df1.head(2).isna().sum().sum()
proc = df1.head(2).isna().sum().sum() / df1.head(2).size
# Вычислим общее количество пропусков в процентах во всей таблице
print('{:.1%}'.format(proc))

93.3%


In [ ]:
# Замена всех пропусков на слово "Нет данных"
df1.fillna("Нет данных", inplace=True)
# Форма и размер
print(df1.shape)
# Колличество элементов
print(df1.size)
df1.head(1)

(32686, 194)
6341084


,КлиентДатаРождения,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Гинекология1_МКБ101,Гинекология1_МКБ102,...,ХирургНеИсп2_МКБ103,ХирургНеИсп3_МКБ101,ХирургНеИсп3_МКБ102,ХирургНеИсп3_МКБ103,ЭКГНеИсп1_МКБ101,ЭКГНеИсп1_МКБ102,ЭКГНеИсп2_МКБ101,ЭКГНеИсп2_МКБ102,Эндоскопия1_МКБ101,Эндоскопия1_МКБ102
0,1989-12-31T00:00:00,Женский,Продавец-кассир,Нет данных,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,Z00.0,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
df1.to_csv('/content/drive/MyDrive/БазыCSV/InClinic/df1.csv', index = False)

# Классификация условий труда



Согласно действующему законодательству, вся трудовая деятельность условно
подразделяется на 4 категории, каждая из которых основывается на степени факторов риска для здоровья и жизни:

---

* Оптимальный — на территории и во внутренних помещениях сохраняется и
поддерживается здоровый микроклимат, положительно влияющий на производительность труда;

* Допустимый — сохраняются нормальные условия, уровень вредных факторов не превышает допустимых норм;

* Вредный — превышаются допустимые нормы, что причиняет вред здоровью человека;

* Опасный — условия труда способны повлечь причинение тяжкого вреда здоровью, иногда несут угрозу жизни.

---

В свою очередь, вредные и опасные производства подразделяются между собой на 4 степени тяжести:

1 Изменения, начавшиеся в организме человека, носят обратимый характер и обычно проявляются после завершения трудовой деятельности. Такие недуги на медицинском сленге называются «профессиональными заболеваниями»;

2 Патологические изменения проявляются более выражено и часто приводят к временной потере трудоспособности (человек регулярно уходит на больничный). Здесь наиболее часто развиваются хронические недуги, вызванные профессиональной деятельностью;

3 В организме протекают необратимые процессы, способные повлечь частичную потерю трудоспособности;

4 Происходят тяжелые функциональные расстройства внутренних органов и систем, что в запущенных случаях приводит к присвоению нерабочей группы инвалидности.


---


К сожалению "СОУТ"(специальная оценка условий труда) пока не располагает системой привязки категории рисков к группам профессии, к примеру ISCO-08, или иные подобные классификаторы.
 Единых методических указаний по управлению профессиональными
рисками пока не существует. Одна из причин - огромное разнообразие профессий и рисков. В одну только Систему стандартов безопасности труда (ССБТ) по состоянию на 2020 год
входит около 400 документов.

---

ISCO-08 МСКЗ-08 делит рабочие места на 10 основных групп:

Основные группы
* 1 Руководители
* 2 Специалисты-профессионалы
* 3 Специалисты-техники и иной средний специальный персонал
* 4 Служащие, занятые подготовкой и оформлением документации, учетом и
  обслуживанием
* 5 Работники сферы обслуживания и торговли
* 6 Квалифицированные работники сельского и лесного хозяйств, рыбоводства и
рыболовства
* 7 Квалифицированные рабочие промышленности и рабочие родственных занятий
* 8 Операторы и сборщики промышленных установок и машин
* 9 Неквалифицированные работники
* 0 Военнослужащие





 Основные группы и подгруппы

* 1 Руководители
11 Руководители высшего звена, высшие должностные лица и законодатели
12 Корпоративные управляющие
13 Руководители подразделений в сфере производства и специализированных
сервисных услуг
14 Руководители в сфере гостиничного бизнеса, руководители (управляющие) в
магазинах и родственных областях деятельности

* 2 Специалисты-профессионалы
21 Специалисты-профессионалы в области науки и техники
22 Специалисты-профессионалы в области здравоохранения
23 Специалисты-профессионалы в области образования
24 Специалисты-профессионалы в сфере бизнеса и администрирования
25 Специалисты-профессионалы по информационно-коммуникационным
технологиям (ИКТ)
26 Специалисты-профессионалы в области права, гуманитарных областей и
культуры

* 3 Специалисты-техники и иной средний специальный персонал
31 Специалисты-техники в области науки и техники
32 Средний медицинский персонал здравоохранения
33 Средний специальный персонал по экономической и административной
деятельности
34 Средний специальный персонал в области юриспруденции, социальной
работы, культуры и родственных занятий
35 Специалисты-техники в области ИКТ

* 4 Служащие, занятые подготовкой и оформлением документации, учетом и
обслуживанием
41 Служащие общего профиля и обслуживающие офисную технику
42 Служащие сферы обслуживания населения
43 Служащие в сфере обработки числовой информации и учета материальных
ценностей
44 Другие офисные служащие

* 5 Работники сферы обслуживания и торговли
51 Работники сфер индивидуальных услуг
52 Продавцы
53 Работники, оказывающие услуги по индивидуальному уходу
54 Работники служб, осуществляющих охрану граждан и собственности

* 6 Квалифицированные работники сельского и лесного хозяйств, рыбоводства и
рыболовства
61 Квалифицированные работники сельского хозяйства, производящие товарную
продукцию
62 Товарные производители лесной и рыбной продукции и охотники
63 Квалифицированные работники сельского хозяйства, рыболовства, охотники и
сборщики урожая, производящие продукцию для личного потребления

* 7 Квалифицированные рабочие промышленности и рабочие родственных занятий
71 Рабочие, занятые в строительстве, и рабочие родственных занятий
72 Рабочие металлообрабатывающей промышленности, машиностроения и
родственных занятий
73 Высококвалифицированные рабочие ручного труда и полиграфического
производства
74 Рабочие в области электротехники и электроники
75 Рабочие пищевой, деревообрабатывающей, текстильной и швейной
промышленности и рабочие родственных занятий

* 8 Операторы и сборщики промышленных установок и машин
81 Операторы промышленных установок и стационарного оборудования
82 Сборщики
83 Водители и операторы подвижного оборудования

* 9 Неквалифицированные работники
91 Прислуга и уборщики
92 Неквалифицированные рабочие сельского и лесного хозяйств, рыбоводства и
рыболовства
93 Неквалифицированные рабочие, занятые в горнодобывающей
промышленности, строительстве, обрабатывающей промышленности и на транспорте
94 Помощники в приготовлении пищи
95 Уличные торговцы и другие неквалифицированные работники, оказывающие
различные уличные услуги
96 Неквалифицированные работники по сбору мусора и другие
неквалифицированные работники

* 0 Военнослужащие
01 Офицеры в действующей армии
02 Военнослужащие неофицерских званий
03 Военнослужащие других званий


# Анализ данных в отобранных столбцах, отбраковка.

По результату анализа данных в отобранных столбцах остаются только столбцы содержащие условно-полноценную информацию небходимую для обучения "НС"

Клиент
- КлиентДатаРождения,  КлиентПол,  Профессия,  ВредныеФакторы.

Обследование
- ПсихОсвидетельствование,  ЗаключениеМК,  ЗаключениеМКПредставление,  МКБ101,  МКБ102,  МКБ103,  МКБ104,  Представление,  ПрофнепригодностьВременнаяПредставление,
 ПрофнепригодностьВременнаяПредставление1, ПрофнепригодностьВременная1, ЖалобыДополнение, Заключение.


---

Запускать только при необходимости анализа

Персона

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'КлиентПол' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['Женский', 'Нет данных', 'Мужской', 'НеУказан']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'Профессия' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
unique_values

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'ВредныеФакторы' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
unique_values

Результаты обследования

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'ПсихОсвидетельствование' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
unique_values

[False, True]

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'ЗаключениеМК' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['ОграниченноГоден', 'тракторист', 'Годен к работе в указанных условиях', 'Годен к работе в указанных условиях (с коррекцией зрения)', ' п. 11.4', 'Годен к работе в указанных условиях (без работ на высотах)', 'ГоденСКоррекциейСлуха', 'Газоспасатель', 'Нет данных', 'ГоденСКоррекциейЗрения', 'Годен к работе в указанных условиях (с коррекцией слуха)', 'Ограниченно годен', ' п. 11.3', 'Годен', 'Нуждается в дообследовании и лечении', 'НуждаетсяВДообследованииИЛечении', 'машинист бульдозера', 'Не годен', 'Негоден', 'ГоденБезРаботНаВысотах', ' Газоспасатель']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'ЗаключениеМКПредставление' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['Годен', 'Не годен', 'Нет данных', 'тракторист', 'Годен к работе в указанных условиях', 'ГоденСКоррекциейЗрения', 'Годен к работе в указанных условиях (с коррекцией зрения)', 'Годен к работе в указанных условиях (с коррекцией слуха)', 'Нуждается в дообследовании и лечении', ' п. 11.4', 'Ограниченно годен', 'Годен к работе в указанных условиях (без работ на высотах)']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "Представление" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
unique_values

['Годен',
 'Не годен',
 'Нет данных',
 'тракторист',
 'Годен к работе в указанных условиях',
 'ГоденСКоррекциейЗрения',
 'Годен к работе в указанных условиях (с коррекцией зрения)',
 'Годен к работе в указанных условиях (с коррекцией слуха)',
 'Нуждается в дообследовании и лечении',
 ' п. 11.4',
 'Ограниченно годен',
 'Годен к работе в указанных условиях (без работ на высотах)']

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "МКБ101" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['D13.0', 'N88.4', 'F06.6', 'L30.9', 'D22.1', 'H11.1', 'Z00.4', 'L40.9', 'E66.0', 'T90.0', 'K29.4', 'D13.4', 'K40.9', 'H18.9', 'H91.9', 'I25.9', 'K20', 'I45.6', 'F10.3', 'O00.0', 'H55', 'N73.9', 'I49.9', 'B35.2', 'E10.8', 'K31.7', 'Z76.9', 'H80.1', 'K76.9', 'H17.9', 'H10.5', 'H65.3', 'S54.0', 'J00', 'N11.9', 'Q44.6', 'N76.1', 'N72', 'H28.2', 'N81.9', 'I47.9', 'D25.9', 'K29.9', 'I69.3', 'G96.9', 'H53.5', 'H10.1', 'K29.7', 'K81.8', 'I48', 'J38.2', 'J38.0', 'Н27', 'K80.1', 'H35.3', 'M05.9', 'H49.9', 'N28.1', 'K82.9', 'E04.1', 'I20.9', 'I67.8', 'Z00.5', 'I48.9', 'B37.8', 'N87.9', 'N85', 'K82.8', 'J35.8', 'K86.0', 'N95.2', 'F10.7', 'D25', 'F06.7', 'I48.0', 'K40.2', 'H16.9', 'H11.9', 'H74.1', 'K00.0', 'K29.3', 'N85.9', 'N60.2', 'T92.4', 'H74.8', 'H40.0', 'M51.1', 'N30.2', 'J45.0', 'T95.8', 'R00.1', 'M17.9', 'H72.9', 'L80.XX', 'Q25.6', 'E03.9', 'I49.8', 'N95.1', 'I12.9', 'F92.9', 'R25.1', 'J31.0', 'H17.8', 'T90.9', 'S02.01', 'H51.9', 'K02.0', 'J34.2', 'J30.2', 'H47.2', 'J37.0', 'F41.0', 'K64.

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "МКБ102" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['C50.1', 'K80.5', 'N83.2', 'H26.9', 'K29.7', 'L23.5', 'K29.6', 'H55', 'H52.2', 'H54.4', 'Q21.1', 'I70.0', 'T90.4', 'F06.8', 'H66.1', 'D35.2', 'D25.0', 'I45.8', 'K82.9', 'H02.4', 'K81.1', 'H90.3', 'E80.4', 'K91.5', 'D25', 'E11.9', 'I49.8', 'H25.0', 'K02.0', 'I05.9', 'J45.8', 'G93.4', 'N30.9', 'G90.8', 'K05.4', 'I77.9', 'K02.1', 'H90.4', 'H25', 'C81.9', 'E11', 'Нет данных', 'H47.2', 'Q13.0', '4.4,5.1,6.1', 'T94.0', 'K29.5', 'I83.9', 'H26.1', 'K29.9', 'M86.6', 'N80.0', 'H52.4', 'C53.9', 'H91.1', 'K76.6', 'I25.8', 'K05.3', 'E11.8', 'J45.9', 'I49.9', 'K86.1', 'H53.0', 'I67.9', 'I34.9', 'J45.0', 'D25.9', 'K80.2', 'F06.6', 'J44.9', 'D16.0', 'F10.3', 'I11.9', 'K25.7', 'C50.4', 'N84.1', 'H74.1', 'Q12.0', 'J42', 'H52.1', 'K29.3', 'M17.9', 'K50.9', 'I72.1', 'F10.2', 'N20.9', 'K44.9', 'N60.0', 'R73.9', 'N11.9', 'H50.0', 'H50.1', 'K04.5', 'I25.0', 'H52.3', 'N30.2', 'L28.0', 'D50.9', 'M42.1', 'K00.0', 'E66.0', 'H17.1', 'C03.0', 'I25.2', 'N28.1', 'H53.9', 'H50.9', 'J33.0', 'I25.9', 'H40.1', 'H34.8',

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "МКБ103" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['I25.8', 'K29.8', 'K76.0', 'K29.6', 'K29.3', 'M05.9', 'M15.9', 'K05.3', 'I06.8', 'N60.2', 'N83.2', 'I25.2', 'Нет данных', 'K80.2', 'K50.9', 'K26.7', 'I67.8', 'H53.0', 'N30.2', 'H66.1', 'K86.1', 'Z00.0', 'H90.3', 'H52.2', 'H52.1', 'D25', 'K81.8', 'H55', 'I11.9', 'H52.0', 'J45.8', 'N30.9', 'K29.5', 'R73.9']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "МКБ104" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

['I25.8', 'K29.6', 'I70.0', 'I06.8', 'N60.2', 'I25.2', 'Нет данных', 'K80.2', 'H53.0', 'K86.1', 'Z00.0', 'H90.3', 'H52.2', 'L20.9', 'I25.9', 'H52.1', 'I11.9', 'H52.0', 'K29.5', 'R73.9']


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "ПрофнепригодностьВременнаяПредставление" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

[]


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "ПрофнепригодностьВременнаяПредставление1" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)

[]


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "ПрофнепригодностьВременная1" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
print(unique_values)



[]


In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if "ЖалобыДополнение" in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
unique_values

[]

In [ ]:
# Уникальные значения колонки
unique_values = []
for column in df1.columns:
    if 'Заключение' in column:
        unique_values.extend(df1[column].unique())
unique_values = list(set(unique_values))
unique_values

# Корректировка колонки'Профессия'

План:

Сделать выборку в колонке 'Профессия' по основному показателю профессии.

Привести список профессий в колонке 'Профессия' к форме применимой для классификации и группировке по типу  МСКЗ-08

Разделение на группировки (Военных не берём в расчёт ) на 9 классов позволит нейронной сети более точно определить соответствующую кореляцию каждой группе и далее кластеризовать требования по наличию потологий для определённой профессии.

Так же можно создать списки категорий профессий по должностям с разделением на степени факторов риска.

Степени факторов риска: Оптимальный,  Допустимый,  Вредный,  Опасный.

Присвоить категории профессии индекс факторов риска, создать новую колонку 'класс безопасности' и заполнить соответствующими значениями индексов.

Создать матрицу корреляции 'Профессия' - 'класс безопасности' - 'ВредныеФакторы'.   Анализ.

Создать матрицу корреляции 'Профессия' - 'класс безопасности' - 'ВредныеФакторы' --- 'Возраст' - 'Пол' - 'Представление' - 'Заключение'.
  Анализ.

Эти мероприятия позволят более точно привязать конкретную профессию и её вредные факторы к персоне со своими факторами - возраст, пол, и наличие патологий и других негативных моментов здоровья.

Всё это позволит определить: подходит ли человек однозначно для выполнения обязанностей или есть необходимость для тщательного выяснения где причина несоответствия и насколько это недопустимо.

В этом и будет заключатся РЕКОМЕНДАЦИЯ.



## Работа над колонкой
Для структурирования данных о профессии и в дальнейшем для классификации и группировке по типу МСКЗ-08 была сделанна выборка всех профессий присутствующих в датасете. Из 32686 записей о профессии клиентов выбранно 624 названия профессии. Для структурирования по намеченному плану необходимо произвести работу по зачистке и исправлению написаний названия, тех же профессий но с иным написанием  названия.Присвоение индекса группы по МСКЗ-08.

Пока работа приостановленна. Нет ресурса времени.  

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/БазыCSV/InClinic/df1.csv')

In [ ]:
df_p = pd.read_excel('/content/drive/MyDrive/БазыCSV/InClinic/Проф1.xlsx')

In [ ]:
df_p.head(3)

In [ ]:
# Создание копии таблицы
df2 =df1.copy()

In [ ]:
df2["Профессия"] = df2["Профессия"].str.lower()  # приведение значений к нижнему регистру
df2 = df2.sort_values("Профессия")  # сортировка по алфавиту

df2.head(3)  # вывод первых 5 строк таблицы


,КлиентДатаРождения,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Гинекология1_МКБ101,Гинекология1_МКБ102,...,ХирургНеИсп2_МКБ103,ХирургНеИсп3_МКБ101,ХирургНеИсп3_МКБ102,ХирургНеИсп3_МКБ103,ЭКГНеИсп1_МКБ101,ЭКГНеИсп1_МКБ102,ЭКГНеИсп2_МКБ101,ЭКГНеИсп2_МКБ102,Эндоскопия1_МКБ101,Эндоскопия1_МКБ102
16396,1968,Мужской,вальщик леса 6 разряда,"4.4,5.1,10,11.1,11.4",False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,Нет данных,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных
20243,1980,Мужской,водитель - механик,2023-01-18 00:00:00,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,Нет данных,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных
19495,1980,Мужской,водитель - механик,2023-01-18 00:00:00,False,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


In [ ]:
dfp = df2[['Профессия']]
# Сохраним колонку 'Профессия' и поработаем в excel
#df2.to_excel('/content/drive/MyDrive/БазыCSV/InClinic/df3.xlsx')

In [ ]:
import re
# приведение значений столбца "Профессия" к нижнему регистру
dfp['Профессия'] = dfp['Профессия'].str.lower()
dfp['вхождения'] = df_p['Проф'].apply(lambda x: ', '.join(re.findall(rf'\b{x}\b', ','.join(dfp['Профессия']))))


<ipython-input-65-14427ec9bc80>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfp['Профессия'] = dfp['Профессия'].str.lower()
<ipython-input-65-14427ec9bc80>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfp['вхождения'] = df_p['Проф'].apply(lambda x: ', '.join(re.findall(rf'\b{x}\b', ','.join(dfp['Профессия']))))


РАБОТА С КОЛОНКОЙ "Профессия" НЕ ЗАКОНЧЕНА

# Корректировка данных

In [ ]:
# Импорт библиотек и инструментов
from google.colab import files              # Импортируем библиотеку files для работы с файлами
import pandas as pd                         # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt             # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import seaborn as sns                       # Импортируем модуль seaborn библиотеки matplotlib для построения графиков
import gdown
import numpy as np
import re
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
                                            # Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/БазыCSV/InClinic/df1.csv')

In [ ]:
# Создание копии таблицы
df3 =df1.copy()

In [ ]:
df3.head(3)

,КлиентДатаРождения,КлиентПол,Профессия,ВредныеФакторы,ПсихОсвидетельствование,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,Гинекология1_МКБ101,Гинекология1_МКБ102,...,ХирургНеИсп2_МКБ103,ХирургНеИсп3_МКБ101,ХирургНеИсп3_МКБ102,ХирургНеИсп3_МКБ103,ЭКГНеИсп1_МКБ101,ЭКГНеИсп1_МКБ102,ЭКГНеИсп2_МКБ101,ЭКГНеИсп2_МКБ102,Эндоскопия1_МКБ101,Эндоскопия1_МКБ102
0,1989-12-31T00:00:00,Женский,Продавец-кассир,Нет данных,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,Z00.0,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных
1,1998-07-20T00:00:00,Женский,Продавец-кассир,2023-10-04 00:00:00,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,Z00.0,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных
2,2002-01-04T00:00:00,Женский,Младший продавец-консультант,23,False,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,Z00.0,Нет данных,...,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных,Нет данных


Колонка возраст

In [ ]:
# Из полной даты рождения нужен толко год возьмём его
df3['КлиентДатаРождения'] = df3['КлиентДатаРождения'].str[:4]

In [ ]:
# Полученный возраст превращаем в класс возрастной категории
def getParameterAgeVect(arg):
  arg = int(arg)  # преобразуем строку в число
  outClass =int((arg-13)/5)
  outClass = max(0, min(4,outClass))
  # На выходе получаем вектор с нужной категорией возраста
  return list(utils.to_categorical(outClass, 5).astype('int'))
# Цикл для прохода по значениям в колонке с годом и вывод результатов
for value in df3.iloc[:3,0]:
    age_c = getParameterAgeVect(value)
    print(f'Год рождения: {value}.')
    print(age_c)

Год рождения: 1989.
[0, 0, 0, 0, 1]
Год рождения: 1998.
[0, 0, 0, 0, 1]
Год рождения: 2002.
[0, 0, 0, 0, 1]


In [ ]:
def getAgeVect(arg):
    arg = int(arg)  # преобразуем строку в число
    # превращаем возраст в категорию возрастной группы
    outClass = int((arg-15)/12)
    outClass = max(0, min(4, outClass))
    # создаем вектор с нужной категорией возраста
    result = np.eye(5)[outClass].astype('int')
    return result

# Цикл для прохода по значениям в колонке с годом и вывод результатов
for value in df3.iloc[:3,0]:
    age_c = getAgeVect(value)
    print(f'Год рождения: {value}')
    print(age_c)

Год рождения: 1989
[0 0 0 0 1]
Год рождения: 1998
[0 0 0 0 1]
Год рождения: 2002
[0 0 0 0 1]


Колонка пол

In [ ]:
# Уникальные значения колонки
display(df3['КлиентПол'].unique())

array(['Женский', 'Мужской', 'Нет данных', 'НеУказан'], dtype=object)

In [ ]:
# заменить 'НеУказан' на 'Нет данных' в столбце 'КлиентПол'
df3['КлиентПол'] = df3['КлиентПол'].replace('НеУказан', 'Нет данных')



In [ ]:
def Sex_class(arg):
    # Создаем список с
    sex_c = [0,0,0]
    if 'М' in arg:
        sex_c = [1,0,0]
    if 'Ж' in arg:
        sex_c = [0,1,0]
    if 'Нет данных' in arg:
        sex_c = [0,0,1]
    # Возвращаем список
    return sex_c
# Проверяем функцию с аргументом 'м'
sex_c = Sex_class('Ж')
# Выводим значение
sex_c

[0, 1, 0]

Колонка Профессия

Колонка Вредные Факторы

In [ ]:
# Уникальные значения колонки
display(df3['ВредныеФакторы'].unique())

array(['Нет данных', '2023-10-04 00:00:00', '23', ...,
       '11.3,11.4,5.1,18.2', '4.4,5.1,6.1,15,17', '4.4,5.1,15,6.1,17'],
      dtype=object)

In [ ]:
# поиск значений с помощью регулярных выражений и удаление найденных записей
df3 = df3[~df3['ВредныеФакторы'].str.match(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}')]

In [ ]:
# Токенизация (разбиение на токены) по запятой
tokenizer = lambda x: x.split(',')
df3['ВредныеФакторы'] = df3['ВредныеФакторы'].apply(tokenizer)

# Преобразование в вектор Bag of Words
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
text_counts = cv.fit_transform(df3['ВредныеФакторы'])

# Преобразование в матрицы нулей и единиц
bow_df = pd.DataFrame(data=text_counts.toarray())

# Сохраняем в массив
arr = bow_df.to_numpy()

In [ ]:
arr[50:56:2]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 

In [ ]:
# токенизация строк по запятой
df3['ВредныеФакторы'] = df3['ВредныеФакторы'].apply(lambda x: x.split(','))

# конвертация в матрицу Bag of Words
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df3['ВредныеФакторы'].apply(lambda x: ', '.join(x)))
x_tr = X.toarray()

In [ ]:
x_tr[50:56:2]


array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
def tokenize_string(s):
    tokens = re.findall(r"[\d\s.,]+", s)
    return [t.strip() for t in tokens]

df3['tokenized'] = df3['ВредныеФакторы'].apply(tokenize_string)
x_Tra = np.array(df3['tokenized'].tolist())


In [ ]:
x_Tra[50:56:2]

array([['5.1,6.1,6.2'],
       ['1.29.2,4.1,4.3.2,11.1,17,18.2,11.4'],
       ['1.30.1,1.50,4.4,6.1,6.2,18.1']], dtype='<U47')

In [ ]:
# токенизируем значения в столбце 'ВредныеФакторы'
df3['tokens'] = df3['ВредныеФакторы'].apply(lambda x: re.findall(r'\d+[,\.]?\d+', x))

# преобразуем список токенов в вектор
df3['vector'] = df3['tokens'].apply(lambda x: [float(i.replace(',', '.')) for i in x])

# выводим результат
print(df3['vector'])


0                             []
2                         [23.0]
3                         [23.0]
4                         [23.0]
5                         [23.0]
                  ...           
32681    [11.3, 11.4, 5.1, 18.2]
32682                [4.2, 5.11]
32683                     [23.0]
32684     [4.4, 5.1, 6.1, 15.17]
32685     [4.4, 5.1, 15.6, 1.17]
Name: vector, Length: 29369, dtype: object


Колонка ПсихОсвидетельствование

In [ ]:
# Уникальные значения колонки
display(df3['ПсихОсвидетельствование'].unique())

array([False,  True])

In [ ]:
def PsExam_class(arg):
    # Создаем список с одним элементом 0
    PsExam = [0]
    # Если True, то значение 1 (по умолчанию 0 для False)
    if 'True' in arg:
        PsExam = 1
    # Возвращаем список
    return PsExam
# Проверяем функцию с аргументом 'м'
PsExam = PsExam_class('False')
# Выводим значение
PsExam

[0]

Колонка'ЗаключениеМК'

In [ ]:
# Уникальные значения колонки
display(df3['ЗаключениеМК'].unique())

array(['Годен', 'ГоденСКоррекциейЗрения',
       'НуждаетсяВДообследованииИЛечении', 'Нет данных',
       'ГоденБезРаботНаВысотах', 'ГоденСКоррекциейСлуха', ' п. 11.3',
       'ОграниченноГоден', 'машинист бульдозера', 'Газоспасатель',
       ' Газоспасатель', 'Негоден'], dtype=object)

In [ ]:
index_list = df3[(df3['ЗаключениеМК'] == 'машинист бульдозера') |
                 (df3['ЗаключениеМК'] == ' Газоспасатель') |
                  (df3['ЗаключениеМК'] == 'Газоспасатель') |
                   (df3['ЗаключениеМК'] == ' п. 11.3') |
                    (df3['ЗаключениеМК'] == 'Нет данных')].index.tolist()


In [ ]:
df3.drop(index_list, inplace=True)

In [ ]:
class_list = ['Годен','НуждаетсяВДообследовании']
def health_sc(df3, column_name, class_list):
    for class_name in class_list:
        count = len(df3[df3[column_name] == class_name])
        print(f'Количество записей для класса {class_name}: {count}')
# количество записей по классам
health_sc(df3, 'ЗаключениеМК', class_list)

# Сокращение классов до 3
df3['ЗаключениеМК'].replace('ГоденСКоррекциейЗрения', 'НуждаетсяВДообследовании', inplace=True)
df3['ЗаключениеМК'].replace('ГоденБезРаботНаВысотах', 'НуждаетсяВДообследовании', inplace=True)
df3['ЗаключениеМК'].replace('НуждаетсяВДообследованииИЛечении', 'НуждаетсяВДообследовании' , inplace=True)
df3['ЗаключениеМК'].replace('Нет данных', 'НуждаетсяВДообследовании', inplace=True)
df3['ЗаключениеМК'].replace('Негоден', 'НуждаетсяВДообследовании', inplace=True)

# Удаление строк без заключения
mask = df3['ЗаключениеМК'].str.contains('|'.join(class_list)) > 0
condition = df3[mask]

# Функция для определения статуса здоровья
def health_stat(condition):
    out = [0, 0]
    if ('Годен' in condition):
        out[0]= 1
    if ('НуждаетсяВДообследовании' in condition):
        out[1]= 1

    # Присваиваем список 'out' переменной 'condition_ohe'
    condition_ohe = out
    return condition_ohe
# Пример вызова функции
health_stat('НуждаетсяВДообследовании')

Количество записей для класса Годен: 17982
Количество записей для класса НуждаетсяВДообследовании: 1209


[0, 1]

ЗаключениеМКПредставление

In [ ]:
# Уникальные значения колонки
display(df3['ЗаключениеМКПредставление'].unique())

array(['Годен к работе в указанных условиях',
       'Годен к работе в указанных условиях (с коррекцией зрения)',
       'Нуждается в дообследовании и лечении', 'Нет данных',
       'Годен к работе в указанных условиях (без работ на высотах)',
       'Годен к работе в указанных условиях (с коррекцией слуха)',
       ' п. 11.4', 'Ограниченно годен', 'тракторист',
       'ГоденСКоррекциейЗрения', 'Годен', 'Не годен'], dtype=object)

In [ ]:
index_list_2 = df3[(df3['ЗаключениеМКПредставление'] == 'тракторист') |
                   (df3['ЗаключениеМКПредставление'] == ' п. 11.4')].index.tolist()

In [ ]:
df3.drop(index_list_2, inplace=True)

In [ ]:
class_list = ['Годен',
              'с коррекцией зрения',
              'без работ на высотах',
              'с коррекцией слуха',
              'Ограниченно годен','Нуждается в дообследовании и лечении']
def health_sp(df3, column_name, class_list):
    for class_name in class_list:
        count = len(df3[df3[column_name] == class_name])
        print(f'Количество записей для класса {class_name}: {count}')
# количество записей по классам
health_sp(df3, 'ЗаключениеМКПредставление', class_list)

# Сокращение классов до 3
df3['ЗаключениеМКПредставление'].replace('Годен к работе в указанных условиях','Годен', inplace=True)
df3['ЗаключениеМКПредставление'].replace('Годен к работе в указанных условиях (с коррекцией зрения)', 'с коррекцией зрения', inplace=True)
df3['ЗаключениеМКПредставление'].replace('Годен к работе в указанных условиях (без работ на высотах)', 'без работ на высотах', inplace=True)
df3['ЗаключениеМКПредставление'].replace('Годен к работе в указанных условиях (с коррекцией слуха)', 'с коррекцией слуха', inplace=True)


# Удаление строк без заключения
mask = df3['ЗаключениеМКПредставление'].str.contains('|'.join(class_list)) > 0
condit = df3[mask]

# Функция для определения статуса здоровья
def health_stp(condit):
    out = [0, 0, 0, 0, 0, 0]
    if ('Годен' in condit):
        out[0]= 1
    if ('с коррекцией зрения' in condit):
        out[1]= 1
    if ('без работ на высотах' in condit):
        out[2]= 1
    if ('с коррекцией слуха' in condit):
        out[3]= 1
    if ('Ограниченно годен' in condit):
        out[4]= 1
    if ('Нуждается в дообследовании и лечении' in condit):
        out[5]= 1

    # Присваиваем список 'out' переменной 'condition_ohe'
    condit_ohe = out
    return condit_ohe
# Пример вызова функции
health_stp('без работ на высотах')

Количество записей для класса Годен: 17984
Количество записей для класса с коррекцией зрения: 1114
Количество записей для класса без работ на высотах: 52
Количество записей для класса с коррекцией слуха: 5
Количество записей для класса Ограниченно годен: 4
Количество записей для класса Нуждается в дообследовании и лечении: 40


[0, 0, 1, 0, 0, 0]

Заключение

In [ ]:
# Уникальные значения колонки
display(df3['Заключение'].unique())

In [ ]:
df3['Заключение'].replace('Годен к работе в указанных условиях','Годен', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях (с коррекцией зрения)', 'с коррекцией зрения', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях (без работ на высотах)', 'без работ на высотах', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях (с коррекцией слуха)', 'с коррекцией слуха', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях с коррекцией зрения', 'с коррекцией зрения', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях, с коррекцией зрения', 'с коррекцией зрения', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях, кроме   ', 'кроме', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях. Противопаказанаработа с подьемом и перемещением тяжестей - более 5 кг, длительной ходьбой, более 500 метров, работа на высоте,в холодных помещениях, эмоциональные нагрузки,полный рабочий день.','Ограниченно годен', inplace=True)
df3['Заключение'].replace('Годен к работе в указанных условиях (без работ на высотах) Противопоказана работа связанная с высотой ,подьем и перемещение тяжестей ,длительной ходьбой.', 'без работ на высотах', inplace=True)

In [ ]:

# создадим новый DataFrame, где будут только строки, в которых больше 7 слов
df_filtered = df3[df3['Заключение'].str.split().apply(len) > 7]

# для строк, где больше 6 слов, возьмем только последние слова
df_filtered['Заключение'] = df_filtered['Заключение'].apply(lambda x: ' '.join(x.split(' ')[6:]))

# объединим отфильтрованные строки с исходным DataFrame
result_df = pd.concat([df_filtered, df3[df3['Заключение'].str.split().apply(len) <= 7]])

# выведем результат
result_df['Заключение'][3::]


In [ ]:
# выборка строк, содержащих скобки в колонке 'Заключение' и удаление скобок
result_df = result_df[result_df['Заключение'].str.contains('\(')]
result_df['Заключение'] = result_df['Заключение'].str.replace('\(|\)', '')

In [ ]:
# Уникальные значения колонки
display(result_df['Заключение'].unique())

In [ ]:
result_df['Заключение']

93         с коррекцией зрения кроме п. 17
115        с коррекцией зрения кроме п.17.
918         с коррекцией зрения кроме п.17
931        с коррекцией зрения кроме п.6.1
933        с коррекцией зрения кроме п.6.1
                       ...                
30626    без работ на высотах кроме п.6.1.
30824     с коррекцией зрения кроме п.6.2.
31756     с коррекцией зрения кроме п.6.1.
31799     с коррекцией зрения кроме п.6.1.
31841     с коррекцией зрения кроме п.6.1.
Name: Заключение, Length: 104, dtype: object

Гинекология1_МКБ101

In [ ]:
# Уникальные значения колонки
display(df3['Гинекология1_МКБ101'].unique())

array(['Z00.0', 'Нет данных', 'N88.0', 'D25', 'Z32.1', 'N95.2', 'D24',
       'N70.9', 'N60.2', 'N95.1', 'H52.1', 'N81.2', 'N84.1', 'N89.4',
       'N80.0', 'I11.9', 'M15.9', 'D25.9', 'K64.3', 'N86', 'O00.0',
       'N90.4', 'D25.0', 'N85', 'N63', 'C50.4', 'N83.2', 'N90.9', 'N91.2',
       'N76.1', 'N81.4', 'N95.3', 'N72', 'O34.0', 'N94.8', 'E22.1',
       'N88.4', 'N30.2', 'N87.9', 'N85.9', 'N73.9', 'N92.6', 'N95.9',
       'N81.9'], dtype=object)

In [ ]:
index_list_2 = df3[(df3['Гинекология1_МКБ101'] == 'Годен к работе в указанных условиях')].index.tolist()

In [ ]:
# токенизация строк по запятой
df3['Гинекология1_МКБ101'] = df3['Гинекология1_МКБ101'].apply(lambda x: x.split(','))

# конвертация в матрицу Bag of Words
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df3['Гинекология1_МКБ101'].apply(lambda x: ', '.join(x)))
x_tr = X.toarray()

In [ ]:
df3.drop(index_list_2, inplace=True)

In [ ]:
df_edit.head(1)

ДАЛЕЕ РАБОТА НЕ ПРОИЗВОДИЛАСЬ

# Формирование классов
Ввиду того что задача создаваемой НС носит характер "Рекомендательной системы"
по этому колонки "ЗаключениеМК" и колонок с "МКБ" для полноценного анализа не достаточно так как данные содержащиеся в этих колонках разбалансированны и мало информативны для  рекомендации.

Есть версия что в колонке "ЗаключениеМК" надо рассматривать два класса "Годен" для однозначных решений, и класс  'НуждаетсяВДообследовании' когда есть наличие каких либо отклонений.  Так как для рекомендации необходимо понимание самой рекомендации необходимо реализовать отсыл к заключениям специалистов которые обнаружили отклонения. При фомировании архитектуры НС для эффективности обучения следует расматривать столбцы с уже приобретёнными болезнями и явными диагнозами такие как
'Заключение', "ЖалобыДополнение" в виде отрицательных факторов 'диагноза', подавая их на отдельный вход вместе с колонками группы персона.

-Так мы устраняем разбалансировку и даём вектор поиска патологий, но метод не будет работать если рассматривать профессии разных групп риска в одном поле.

-Для более точной работы "НС" к примеру не желательно рассматривать музыканта в одном поле с шахтёром так как разные подходы к требованиям состояний здоровья, для этого и понадобится классификация групп профессий с кластеризацией по факторам риска.

In [ ]:
# Импорт библиотек и инструментов
from google.colab import files              # Импортируем библиотеку files для работы с файлами
import pandas as pd                         # Импортируем библиотеку обработки и анализа данных pandas
import matplotlib.pyplot as plt             # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
import seaborn as sns                       # Импортируем модуль seaborn библиотеки matplotlib для построения графиков
import gdown
import numpy as np
import re
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

In [ ]:
# Создание копии таблицы
health_survey_backup = health_survey.copy()
health_survey.head(1)

ДАЛЕЕ РАБОТА НЕ ПРОИЗВОДИЛАСЬ

# Формирование выборок для обучения

In [ ]:
# Функция для получения обучающих данных в формате 0/1
def get_01_data(data):
    x_train = []
    y_train = []

    for val in data:
        y = health_stat(val)
        x = get_all_parameters(val)
        x_train.append(x)
        y_train.append(y)

    x_train = np.array(x_train)
    y_train = np.array(y_train)

    return x_train, y_train

x_train, y_train = get_01_data(health_survey.values)

In [ ]:
health_survey_subset = health_survey.iloc[:, [1,2,3,4,5]] # Выбор столбцов с нужными индексами
text_data = health_survey_subset.to_string(header=False, index=False) # Преобразование в строку
tokens = text_data.lower().split() # Приведение к нижнему регистру и токенизация


In [ ]:
# Создание экземпляра класса CountVectorizer
vectorizer = CountVectorizer()

# Преобразование токенов в матрицу Bag of Words
bow_matrix = vectorizer.fit_transform(tokens)

# Преобразование в разреженный массив NumPy
sparse_bow_matrix = bow_matrix.toarray()


In [ ]:
print(sparse_bow_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
y_data_01, y_train_01 = Y_to_OHE(dffix5_cut)
print (y_data_01[100])
print (y_train_01[100])

y = np.argmax(y_train_01, axis=-1)
print (y[100])

2
[0. 0. 1.]
2


# Эксперименты обучения

In [ ]:
health_stat = pd.read_csv('/content/health_stat.csv')

In [ ]:
from tensorflow.keras import utils
loss, acc = search.evaluate(x1_test, y_test, verbose=0) #classification
mae, _ = search.evaluate(x1_test, y_test, verbose=0) #regression
y_predictions = search.predict(x1_test)
model = search.export_model()
model.summary()
utils.plot_model(model, show_shapes=True, show_layer_names=True, dpi=60, show_layer_activations=True, show_trainable=True)

In [ ]:
# Обучаем модель полученными данными
model = Sequential()
model.add(BatchNormalization(input_shape=(xTrain01.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1000, activation='tanh'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=1e-5), loss='mse', metrics=['mae'])

history = model.fit(xTrain01,
                    yTrain,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.15,
                    verbose=2)

plt.plot(history.history['mae'],
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_mae'],
         label='Средняя абсолютная ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

In [ ]:
import autokeras as ak
search = ak.StructuredDataClassifier(max_trials=15)
search.fit(x=x1_train, y=y_train, verbose=1)

In [ ]:
scores2 = model_final.evaluate([x1_val, x2_val, x3_val],
                        y_val,
                        verbose=1
                        )
plt.plot(history.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')

# Визуализация точности на проверочной выборке
plt.plot(history.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')

# Отрисовка подписей осей
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')

# Отрисовка легенды
plt.legend()

# Вывод графика
plt.show()


plt.plot(history.history['loss'],
         label='Ошибка на обучающем наборе')

plt.plot(history.history['val_loss'],
         label='Ошибка на проверочном наборе')

plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')

plt.legend()

plt.show()